# PyTorch: Using MLflow and Optuna for experiment tracking and hyperparameter optimization

## Imports

In [9]:
import optuna
import mlflow
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from mlflow import pytorch
from pprint import pformat

/Users/sstoppel/miniconda3/envs/test-opt/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Network 

In [10]:
class Net(nn.Module):
    def __init__(self, dropout=0.0):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(dropout)
        self.dropout2 = nn.Dropout2d(dropout)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
    

## Train and validation functions

In [11]:
# Training loop
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_set_size = len(train_loader.dataset)
    num_batches = len(train_loader)
    train_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            batch_size = len(data)
            print(f"Train Epoch: {epoch} [{batch_idx * batch_size}/{train_set_size} "
                  f"({100. * batch_idx / num_batches:.0f}%)]\tLoss: {loss.item():.6f}")
    avg_train_loss = train_loss / num_batches
    return avg_train_loss

# Testing loop
def validate(model, device, val_loader):
    model.eval()
    val_set_size = len(val_loader.dataset)
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    val_loss /= val_set_size

    print(f"Test set: Average loss: {val_loss:.4f}, Accuracy: {correct}/{val_set_size} "
          f"({100. * correct / val_set_size:.0f}%)\n")
    return val_loss

## Main

In [12]:

# Obtain hyperparameters for this trial
def suggest_hyperparameters(trial):
    # Obtain the learning rate on a logarithmic scale
    lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
    # Obtain the dropout ratio in a range from 0.0 to 0.9 with step size 0.1
    dropout = trial.suggest_float("dropout", 0.0, 0.9, step=0.1)
    # Obtain the optimizer to use by name
    optimizer_name = trial.suggest_categorical("optimizer_name", ["Adam", "Adadelta"])

    print(f"Suggested hyperparameters: \n{pformat(trial.params)}")
    return lr, dropout, optimizer_name

In [13]:
def get_mnist_dataloaders(batch_size=8):
    # Load the MNIST train and test datasets and save them to ./data
    mnist_train = datasets.MNIST('./data', train=True, download=True, transform=transforms.Compose([
                                               transforms.ToTensor(),
                                               transforms.Normalize((0.1307,), (0.3081,))
                                           ]))
    train_loader = torch.utils.data.DataLoader(mnist_train,
                                                       batch_size=batch_size,
                                                       shuffle=True)
    mnist_test = datasets.MNIST('./data', train=False, download=True, transform=transforms.Compose([
                                               transforms.ToTensor(),
                                               transforms.Normalize((0.1307,), (0.3081,))
                                           ]))
    val_loader = torch.utils.data.DataLoader(mnist_test,
                                                      batch_size=1000,
                                                      shuffle=True)
    return train_loader, val_loader

In [14]:
def objective(trial):
    print("\n********************************\n")
    best_val_loss = float('Inf')
    
    # Start a new mlflow run
    with mlflow.start_run():
        # Get hyperparameter suggestions created by optuna and log them as params using mlflow
        lr, dropout, optimizer_name = suggest_hyperparameters(trial)
        mlflow.log_params(trial.params)

        # Use CUDA if GPU is available and log device as param using mlflow
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        mlflow.log_param("device", device)

        # Initialize network
        model = Net(dropout=dropout).to(device)

        # Pick an optimizer based on optuna's parameter suggestion
        if optimizer_name == "Adam":
            optimizer = optim.Adam(model.parameters(), lr=lr)
        if optimizer_name == "Adadelta":
            optimizer = optim.Adadelta(model.parameters(), lr=lr)
        scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
        
        # Get DataLoaders for MNIST train and validation set
        train_loader, val_loader = get_mnist_dataloaders()
        
        # Network training & validation loop
        for epoch in range(0, 5):
            avg_train_loss = train(model, device, train_loader, optimizer, epoch)
            avg_val_loss = validate(model, device, val_loader)
            
            if avg_val_loss <= best_val_loss:
                best_val_loss = avg_val_loss

            # Log average train and validation set loss metrics for the current epoch using mlflow
            mlflow.log_metric("avg_train_losses", avg_train_loss, step=epoch)
            mlflow.log_metric("avg_val_loss", avg_val_loss, step=epoch)
            
            scheduler.step()

    # Return the best validation loss achieved by the network.
    # This is needed as Optuna needs to know how the suggested hyperparameters are influencing the network loss.
    return best_val_loss

In [15]:
def main():
    # Create the optuna study which shares the experiment name
    study = optuna.create_study(study_name="pytorch-mlflow-optuna", direction="minimize")
    study.optimize(objective, n_trials=5)

    # Print optuna study statistics
    print("\n++++++++++++++++++++++++++++++++++\n")
    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Trial number: ", trial.number)
    print("  Loss (trial value): ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))


In [ ]:
main()


[I 2020-11-16 12:11:06,622] A new study created in memory with name: pytorch-mlflow-optuna



********************************

Suggested hyperparameters: 
{'dropout': 0.1, 'lr': 0.001615819736728719, 'optimizer_name': 'Adadelta'}
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.260692
Train Epoch: 0 [80/60000 (0%)]	Loss: 2.231736
Train Epoch: 0 [160/60000 (0%)]	Loss: 2.332082
Train Epoch: 0 [240/60000 (0%)]	Loss: 2.354262
Train Epoch: 0 [320/60000 (1%)]	Loss: 2.359829
Train Epoch: 0 [400/60000 (1%)]	Loss: 2.338420
Train Epoch: 0 [480/60000 (1%)]	Loss: 2.279206
Train Epoch: 0 [560/60000 (1%)]	Loss: 2.268904
Train Epoch: 0 [640/60000 (1%)]	Loss: 2.345856
Train Epoch: 0 [720/60000 (1%)]	Loss: 2.248383
Train Epoch: 0 [800/60000 (1%)]	Loss: 2.278030
Train Epoch: 0 [880/60000 (1%)]	Loss: 2.224254
Train Epoch: 0 [960/60000 (2%)]	Loss: 2.302792
Train Epoch: 0 [1040/60000 (2%)]	Loss: 2.215610
Train Epoch: 0 [1120/60000 (2%)]	Loss: 2.259586
Train Epoch: 0 [1200/60000 (2%)]	Loss: 2.305805
Train Epoch: 0 [1280/60000 (2%)]	Loss: 2.220117
Train Epoch: 0 [1360/60000 (2%)]	Loss: 2.184291
Train Epoch: 0

Train Epoch: 0 [13280/60000 (22%)]	Loss: 0.697360
Train Epoch: 0 [13360/60000 (22%)]	Loss: 1.059415
Train Epoch: 0 [13440/60000 (22%)]	Loss: 0.886676
Train Epoch: 0 [13520/60000 (23%)]	Loss: 1.084192
Train Epoch: 0 [13600/60000 (23%)]	Loss: 1.064804
Train Epoch: 0 [13680/60000 (23%)]	Loss: 1.315719
Train Epoch: 0 [13760/60000 (23%)]	Loss: 1.133237
Train Epoch: 0 [13840/60000 (23%)]	Loss: 0.744685
Train Epoch: 0 [13920/60000 (23%)]	Loss: 0.743803
Train Epoch: 0 [14000/60000 (23%)]	Loss: 0.857451
Train Epoch: 0 [14080/60000 (23%)]	Loss: 1.092209
Train Epoch: 0 [14160/60000 (24%)]	Loss: 0.875712
Train Epoch: 0 [14240/60000 (24%)]	Loss: 1.041354
Train Epoch: 0 [14320/60000 (24%)]	Loss: 0.855162
Train Epoch: 0 [14400/60000 (24%)]	Loss: 0.708860
Train Epoch: 0 [14480/60000 (24%)]	Loss: 0.926127
Train Epoch: 0 [14560/60000 (24%)]	Loss: 0.927099
Train Epoch: 0 [14640/60000 (24%)]	Loss: 0.767822
Train Epoch: 0 [14720/60000 (25%)]	Loss: 0.969886
Train Epoch: 0 [14800/60000 (25%)]	Loss: 1.026645


Train Epoch: 0 [26400/60000 (44%)]	Loss: 0.412189
Train Epoch: 0 [26480/60000 (44%)]	Loss: 0.470729
Train Epoch: 0 [26560/60000 (44%)]	Loss: 1.075625
Train Epoch: 0 [26640/60000 (44%)]	Loss: 0.478368
Train Epoch: 0 [26720/60000 (45%)]	Loss: 0.651349
Train Epoch: 0 [26800/60000 (45%)]	Loss: 0.360883
Train Epoch: 0 [26880/60000 (45%)]	Loss: 0.394133
Train Epoch: 0 [26960/60000 (45%)]	Loss: 0.401739
Train Epoch: 0 [27040/60000 (45%)]	Loss: 0.636483
Train Epoch: 0 [27120/60000 (45%)]	Loss: 1.414557
Train Epoch: 0 [27200/60000 (45%)]	Loss: 1.384901
Train Epoch: 0 [27280/60000 (45%)]	Loss: 0.751790
Train Epoch: 0 [27360/60000 (46%)]	Loss: 1.152703
Train Epoch: 0 [27440/60000 (46%)]	Loss: 0.320355
Train Epoch: 0 [27520/60000 (46%)]	Loss: 0.323871
Train Epoch: 0 [27600/60000 (46%)]	Loss: 0.345725
Train Epoch: 0 [27680/60000 (46%)]	Loss: 0.364117
Train Epoch: 0 [27760/60000 (46%)]	Loss: 1.080735
Train Epoch: 0 [27840/60000 (46%)]	Loss: 0.544384
Train Epoch: 0 [27920/60000 (47%)]	Loss: 0.691999


Train Epoch: 0 [39520/60000 (66%)]	Loss: 0.156803
Train Epoch: 0 [39600/60000 (66%)]	Loss: 0.698271
Train Epoch: 0 [39680/60000 (66%)]	Loss: 0.509034
Train Epoch: 0 [39760/60000 (66%)]	Loss: 0.277964
Train Epoch: 0 [39840/60000 (66%)]	Loss: 1.539281
Train Epoch: 0 [39920/60000 (67%)]	Loss: 0.271271
Train Epoch: 0 [40000/60000 (67%)]	Loss: 0.507041
Train Epoch: 0 [40080/60000 (67%)]	Loss: 0.115138
Train Epoch: 0 [40160/60000 (67%)]	Loss: 0.034043
Train Epoch: 0 [40240/60000 (67%)]	Loss: 0.703041
Train Epoch: 0 [40320/60000 (67%)]	Loss: 0.432563
Train Epoch: 0 [40400/60000 (67%)]	Loss: 0.356181
Train Epoch: 0 [40480/60000 (67%)]	Loss: 0.402149
Train Epoch: 0 [40560/60000 (68%)]	Loss: 0.761136
Train Epoch: 0 [40640/60000 (68%)]	Loss: 0.355467
Train Epoch: 0 [40720/60000 (68%)]	Loss: 0.093634
Train Epoch: 0 [40800/60000 (68%)]	Loss: 0.369354
Train Epoch: 0 [40880/60000 (68%)]	Loss: 0.537264
Train Epoch: 0 [40960/60000 (68%)]	Loss: 0.391723
Train Epoch: 0 [41040/60000 (68%)]	Loss: 0.537256


...

Test set: Average loss: 0.2334, Accuracy: 9301/10000 (93%)


********************************

Suggested hyperparameters: 
{'dropout': 0.5, 'lr': 0.022053063057492, 'optimizer_name': 'Adadelta'}
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.367640
Train Epoch: 0 [80/60000 (0%)]	Loss: 2.278614
Train Epoch: 0 [160/60000 (0%)]	Loss: 2.247780
Train Epoch: 0 [240/60000 (0%)]	Loss: 2.256153
Train Epoch: 0 [320/60000 (1%)]	Loss: 2.083424
Train Epoch: 0 [400/60000 (1%)]	Loss: 2.197701
Train Epoch: 0 [480/60000 (1%)]	Loss: 2.237154
Train Epoch: 0 [560/60000 (1%)]	Loss: 2.174597
Train Epoch: 0 [640/60000 (1%)]	Loss: 2.128475
Train Epoch: 0 [720/60000 (1%)]	Loss: 2.052453
Train Epoch: 0 [800/60000 (1%)]	Loss: 2.046446
Train Epoch: 0 [880/60000 (1%)]	Loss: 2.091293
Train Epoch: 0 [960/60000 (2%)]	Loss: 1.949594
Train Epoch: 0 [1040/60000 (2%)]	Loss: 2.222526
Train Epoch: 0 [1120/60000 (2%)]	Loss: 1.928779
Train Epoch: 0 [1200/60000 (2%)]	Loss: 1.735315
Train Epoch: 0 [1280/60000 (2%)]	Loss: 1.549660
Train

Train Epoch: 0 [13200/60000 (22%)]	Loss: 0.453564
Train Epoch: 0 [13280/60000 (22%)]	Loss: 0.082530
Train Epoch: 0 [13360/60000 (22%)]	Loss: 0.885698
Train Epoch: 0 [13440/60000 (22%)]	Loss: 0.153031
Train Epoch: 0 [13520/60000 (23%)]	Loss: 0.785636
Train Epoch: 0 [13600/60000 (23%)]	Loss: 0.240776
Train Epoch: 0 [13680/60000 (23%)]	Loss: 0.086893
Train Epoch: 0 [13760/60000 (23%)]	Loss: 1.381331
Train Epoch: 0 [13840/60000 (23%)]	Loss: 0.505790
Train Epoch: 0 [13920/60000 (23%)]	Loss: 0.315462
Train Epoch: 0 [14000/60000 (23%)]	Loss: 0.099969
Train Epoch: 0 [14080/60000 (23%)]	Loss: 0.608726
Train Epoch: 0 [14160/60000 (24%)]	Loss: 0.812697
Train Epoch: 0 [14240/60000 (24%)]	Loss: 0.765353
Train Epoch: 0 [14320/60000 (24%)]	Loss: 0.649567
Train Epoch: 0 [14400/60000 (24%)]	Loss: 0.205792
Train Epoch: 0 [14480/60000 (24%)]	Loss: 1.138540
Train Epoch: 0 [14560/60000 (24%)]	Loss: 1.003811
Train Epoch: 0 [14640/60000 (24%)]	Loss: 0.427049
Train Epoch: 0 [14720/60000 (25%)]	Loss: 0.033859


Train Epoch: 0 [26320/60000 (44%)]	Loss: 1.169208
Train Epoch: 0 [26400/60000 (44%)]	Loss: 0.315170
Train Epoch: 0 [26480/60000 (44%)]	Loss: 0.256128
Train Epoch: 0 [26560/60000 (44%)]	Loss: 1.004811
Train Epoch: 0 [26640/60000 (44%)]	Loss: 0.407610
Train Epoch: 0 [26720/60000 (45%)]	Loss: 0.332844
Train Epoch: 0 [26800/60000 (45%)]	Loss: 0.205814
Train Epoch: 0 [26880/60000 (45%)]	Loss: 0.916602
Train Epoch: 0 [26960/60000 (45%)]	Loss: 0.244793
Train Epoch: 0 [27040/60000 (45%)]	Loss: 0.256661
Train Epoch: 0 [27120/60000 (45%)]	Loss: 0.907168
Train Epoch: 0 [27200/60000 (45%)]	Loss: 0.148430
Train Epoch: 0 [27280/60000 (45%)]	Loss: 0.210693
Train Epoch: 0 [27360/60000 (46%)]	Loss: 0.036390
Train Epoch: 0 [27440/60000 (46%)]	Loss: 0.085804
Train Epoch: 0 [27520/60000 (46%)]	Loss: 0.132835
Train Epoch: 0 [27600/60000 (46%)]	Loss: 0.466856
Train Epoch: 0 [27680/60000 (46%)]	Loss: 0.878834
Train Epoch: 0 [27760/60000 (46%)]	Loss: 0.034243
Train Epoch: 0 [27840/60000 (46%)]	Loss: 0.952749


Train Epoch: 0 [39440/60000 (66%)]	Loss: 0.038071
Train Epoch: 0 [39520/60000 (66%)]	Loss: 0.137876
Train Epoch: 0 [39600/60000 (66%)]	Loss: 0.119808
Train Epoch: 0 [39680/60000 (66%)]	Loss: 0.136899
Train Epoch: 0 [39760/60000 (66%)]	Loss: 0.079457
Train Epoch: 0 [39840/60000 (66%)]	Loss: 0.054341
Train Epoch: 0 [39920/60000 (67%)]	Loss: 0.163622
Train Epoch: 0 [40000/60000 (67%)]	Loss: 0.950898
Train Epoch: 0 [40080/60000 (67%)]	Loss: 0.084915
Train Epoch: 0 [40160/60000 (67%)]	Loss: 0.742626
Train Epoch: 0 [40240/60000 (67%)]	Loss: 0.346340
Train Epoch: 0 [40320/60000 (67%)]	Loss: 0.043206
Train Epoch: 0 [40400/60000 (67%)]	Loss: 0.365891
Train Epoch: 0 [40480/60000 (67%)]	Loss: 0.196454
Train Epoch: 0 [40560/60000 (68%)]	Loss: 1.013704
Train Epoch: 0 [40640/60000 (68%)]	Loss: 0.198607
Train Epoch: 0 [40720/60000 (68%)]	Loss: 0.356548
Train Epoch: 0 [40800/60000 (68%)]	Loss: 0.398653
Train Epoch: 0 [40880/60000 (68%)]	Loss: 0.106808
Train Epoch: 0 [40960/60000 (68%)]	Loss: 0.791338


Train Epoch: 0 [52560/60000 (88%)]	Loss: 0.187022
Train Epoch: 0 [52640/60000 (88%)]	Loss: 0.231587
Train Epoch: 0 [52720/60000 (88%)]	Loss: 0.072582
Train Epoch: 0 [52800/60000 (88%)]	Loss: 0.136090
Train Epoch: 0 [52880/60000 (88%)]	Loss: 0.693186
Train Epoch: 0 [52960/60000 (88%)]	Loss: 0.487460
Train Epoch: 0 [53040/60000 (88%)]	Loss: 0.188951
Train Epoch: 0 [53120/60000 (89%)]	Loss: 0.095453
Train Epoch: 0 [53200/60000 (89%)]	Loss: 0.106623
Train Epoch: 0 [53280/60000 (89%)]	Loss: 0.910972
Train Epoch: 0 [53360/60000 (89%)]	Loss: 0.367102
Train Epoch: 0 [53440/60000 (89%)]	Loss: 0.022474
Train Epoch: 0 [53520/60000 (89%)]	Loss: 0.054662
Train Epoch: 0 [53600/60000 (89%)]	Loss: 1.050424
Train Epoch: 0 [53680/60000 (89%)]	Loss: 0.296759
Train Epoch: 0 [53760/60000 (90%)]	Loss: 0.188044
Train Epoch: 0 [53840/60000 (90%)]	Loss: 0.136359
Train Epoch: 0 [53920/60000 (90%)]	Loss: 0.153635
Train Epoch: 0 [54000/60000 (90%)]	Loss: 0.507412
Train Epoch: 0 [54080/60000 (90%)]	Loss: 0.029177
